In [ ]:
import os
import json
import torch

from json import JSONDecodeError
import numpy as np
from tqdm import tqdm
from PIL import Image

from transformers import ViTForImageClassification, ViTFeatureExtractor

from collections import defaultdict

from util.constants import Topic, ThumbnailURL, thumbnail_URL
import requests

In [ ]:
# Models

class ImageLatentRepresentationModel(ViTForImageClassification):
    """
    Hook into the ViTForImageClassification Class in order to get the latent
    representations of an image, not just the classification output. Source code
    taken from HuggingFace open-source GitHub:
    https://github.com/huggingface/transformers/blob/main/src/transformers/models/vit/modeling_vit.py
    """

    def __init__(self, config):
        super().__init__(config)

    def forward(self, pixel_values):
        """
        Overwritten forward method to only get latent representation of the image,
        without image classification.

        args:
            - pixel_values: input image, as PyTorch Tensor of shape [1,3,224,224] 
        
        returns:
            - latent_vec: latent vector representing the image
        """
        vit_output = self.vit(pixel_values)
        latent_vec = vit_output[0][:,0,:]

        return latent_vec

def load_model(device, install=True):
    """
    Loads in a pre-trained ViT model for latent image representation

    args:
        - device: what device the model should be on
    returns:
        - model: pre-trained ViT model
        - feature_extractor: pre-trained feature extractor for processing images
    """
    if install:
        print("Installing ViT architecture... This may take a couple of minutes.")
        os.system("pip install -q git+https://github.com/huggingface/transformers.git")
        print("Finished installing.")

    print("Loading pretrained ViT model...")
    model = ImageLatentRepresentationModel.from_pretrained('google/vit-base-patch16-224')
    model.eval()
    model.to(device)
    print("Loaded model")

    feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

    return model, feature_extractor

def get_latent_vectors(model, ft_extr, raw_imgs, device):
    """
    Function to get the latent representation of an image.

    args:
        - model: The pretrained ViT model
        - ft_extr: The feature extractor, used to preprocess the images
        - raw_imgs: The raw thumbnail images
    """
    # Encode the images using the feature extractor
    encodings = ft_extr(images=raw_imgs, return_tensors="pt")
    pixel_values = encodings['pixel_values'].to(device)
#     print(f"Size of images in memory: {pixel_values.element_size()*pixel_values.nelement()} bytes")

    # Get the latent representation by passing it through the network
    latent_vecs = model(pixel_values)
    del pixel_values
    return latent_vecs

def generate_repr_stats(model, feature_extractor, video_thumbnails):
    """
    Function to generate all thumbnail latent representation statistics.

    args:
        - model: pre-trained ViT model
        - feature_extractor: pre-trained feature extractor for processing images
        - video_thumbnails: images we want to represent
    """
    latents = get_latent_vectors(model, feature_extractor, video_thumbnails, device)
    latents_save = latents.detach().cpu().numpy().tolist()
    del latents
    return latents_save

Get latents for all videos

In [ ]:
# Read data
videos = []
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        videos_info = json.load(f)
        videos.extend([vid for channel_vids in videos_info.values() for vid in channel_vids])

In [ ]:
RESULTS_DIR = os.path.join("..", "data", "thumbnail-latents")

video_results_dir = os.path.join("..","..","DATA","thumbnail-latents","videos")
channel_results_dir = os.path.join(RESULTS_DIR, "channels")
categories_results_dir = os.path.join(RESULTS_DIR, "categories")

if not os.path.exists(video_results_dir):
    os.makedirs(video_results_dir)

if not os.path.exists(categories_results_dir):
    os.makedirs(categories_results_dir)

def get_done_list(dir):
    return [nm.replace(".json",'').replace('.pt','') for nm in os.listdir(dir)]

In [ ]:
# Run the code in batches
done_list = get_done_list(video_results_dir)
all_ids = [vid["id"] for vid in tqdm(videos)]
todo_ids = list(set(all_ids).difference(done_list))

THUMBNAIL_DIR = os.path.join("..","data","thumbnails")
quality = ThumbnailURL.high

FETCH_THUMBS = True
if not FETCH_THUMBS:
    available_ids = [nm.replace('_high.jpg','') for nm in get_done_list(THUMBNAIL_DIR)]
    todo_ids = list(set(todo_ids).intersection(available_ids))

batch_size = 16
batch_num = len(todo_ids)//batch_size
if batch_num != int(len(todo_ids)/batch_size):
    batch_num += 1


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}\n")

model, feature_extractor = load_model(device, install=False)

for batch in tqdm(range(batch_num)):
    ids = todo_ids[batch*batch_size:(batch+1)*batch_size]

    if FETCH_THUMBS:
        raws = []
        fetched_ids = []
        for id in ids:
            url = thumbnail_URL(id, quality)
            try:
                raws.append(requests.get(url, stream=True).raw)
            except:
                continue
            fetched_ids.append(id)
        images = [Image.open(raw) for raw in raws]
        ids = fetched_ids
    else:
        imgs_paths = [os.path.join(THUMBNAIL_DIR, id+"_high.jpg") for id in ids]
        images = [Image.open(path) for path in imgs_paths]

    batch_latents = generate_repr_stats(model, feature_extractor, images)

    for vid_id, result in zip(ids, batch_latents):
        path = os.path.join(video_results_dir, f"{vid_id}.pt")
        torch.save(result, path)

    torch.cuda.empty_cache()

Channel stats

In [ ]:
# Read data
with open(os.path.join("..", "data", "vid2channel.json"), "r") as f:
    vid2channel = json.load(f)

In [ ]:
# Make list of results per video for each channel
channel_result_list = defaultdict(list)
for vid_id in tqdm(get_done_list(video_results_dir)):
    channel = vid2channel[vid_id]
    filepath = os.path.join(video_results_dir, f"{vid_id}.json")
    try:
        with open(filepath, "r") as f:
            result = json.load(f)
    except JSONDecodeError:
        print(f"couldn't open {vid_id}; deleting file")
        os.remove(filepath)
    channel_result_list[channel].append(result)

In [ ]:
# Calculate channel results
for channel,result_list in channel_result_list.items():
    result_list = np.array(result_list)
    channel_result = {
        "mean": result_list.mean(axis=0).tolist(),
        "std": result_list.std(),
        "len": len(result_list),
    }

    filepath = os.path.join(channel_results_dir, f"{channel}.json")
    with open(filepath, "w") as f:
        json.dump(channel_result, f)

Category stats

In [ ]:
# Read data
with open(os.path.join("..", "data", "channel2category.json"), "r") as f:
    channel2cat = json.load(f)

In [ ]:
# Make list of results per channel for each category
category_results_list = defaultdict(list)
for channel in tqdm(get_done_list(channel_results_dir)):
    cat = channel2cat[channel]
    filepath = os.path.join(channel_results_dir, f"{channel}.json")
    try:
        with open(filepath, "r") as f:
            results = json.load(f)
    except JSONDecodeError:
        print(f"couldn't open {channel}; deleting file")
        os.remove(filepath)
    category_results_list[cat].append(results)

In [ ]:
# Calculate category results
for cat,stats_list in category_results_list.items():
    mean_list = np.array([channel_stats["mean"] for channel_stats in stats_list])
    std_list = np.array([channel_stats["std"] for channel_stats in stats_list])
    category_result = {
        "mean": mean_list.mean(axis=0).tolist(),
        "std": std_list.mean(),
        "len": len(mean_list),
    }

    filepath = os.path.join(RESULTS_DIR, "categories", f"{cat}.json")
    with open(filepath, "w") as f:
        json.dump(category_result, f)